## B mode 불량 이미지 검출 (B_normal에서 ResNet 모델 만들기)
### grayscale -> 3 channel
### 정규화 0~1

In [2]:
# 필요한 라이브러리
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

# 데이터셋 불러오기
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# 데이터 분할
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

# 모델 정의
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models

## 데이터셋 불러오기

In [3]:
import os
from PIL import Image

data_path = "D:/사용자/Jina/AI_image/B"
subfolders = ["Broken", "Normal"]

for subfolder in subfolders:
    subfolder_path = os.path.join(data_path, subfolder)
    for filename in os.listdir(subfolder_path):
        file_path = os.path.join(subfolder_path, filename)
        with Image.open(file_path) as img:
            if img.size != (256, 256):
                img = img.resize((256, 256))


In [15]:
# 데이터셋 불러오기
dataset = datasets.ImageFolder( 
    root=data_path,
    transform=transforms.Compose([
        transforms.Resize((256,256)),  # 이미지 크기 조정
        transforms.ToTensor(),          # 이미지를 텐서로 변환
        transforms.Normalize((0.0,), (1.0,)) # 정규화, (0.5,), (0.5,) -> 흑백 이미지이기에 (0.0,), (1.0,)
    ]),
    # 클래스를 0, 1로 변경
    target_transform=lambda x: 1 if x == 1 else 0 if x == 0 else None
)


In [16]:
# 이미지 데이터셋
dataset

Dataset ImageFolder
    Number of datapoints: 5806
    Root location: D:/사용자/Jina/AI_image/B
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.0,), std=(1.0,))
           )
Target transform: <function <lambda> at 0x000002290E9D1940>

In [6]:
len(dataset)

5806

In [17]:
# 데이터셋 분할
train_size = int(0.7 * len(dataset))
test_size = int(0.2 * len(dataset))
val_size = len(dataset) - train_size - test_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# DataLoader로 데이터셋을 배치 단위로 로드
batch_size = 16
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [18]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))


4064
581
1161


In [19]:
train_dataset[0]

(tensor([[[0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          ...,
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627]],
 
         [[0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          ...,
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627]],
 
         [[0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ..., 0.0627, 0.0627, 0.0627],
          [0.0627, 0.0627, 0.0627,  ...,

## 모델 정의

In [20]:
# ResNet-18 모델 정의
model = models.resnet18(pretrained=True)

# 모델 뒤에 fully connected layer 추가
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 이진 분류 문제이므로

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [22]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 손실함수, 옵티마이저

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## 모델 학습

In [24]:
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print('Epoch %d, loss: %.5f' % (epoch + 1, running_loss / len(train_loader)))

Epoch 1, loss: 0.07402
Epoch 2, loss: 0.00772
Epoch 3, loss: 0.00324
Epoch 4, loss: 0.00420
Epoch 5, loss: 0.00215
Epoch 6, loss: 0.00248
Epoch 7, loss: 0.00092
Epoch 8, loss: 0.00071
Epoch 9, loss: 0.00112
Epoch 10, loss: 0.00050


In [ ]:
# 6월 13일, 10:55분  Epoch 1, loss: 0.06824

## 모델 평가

In [12]:
def evaluate(model, criterion, val_loader):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
    
    val_loss = running_loss / len(val_loader.dataset)
    val_acc = running_corrects.double() / len(val_loader.dataset)
    return val_loss, val_acc

# 모델 평가
val_loss, val_acc = evaluate(model, criterion, val_loader)
print('Validation Loss: {:.5f} | Validation Accuracy: {:.5f}'.format(val_loss, val_acc))


Validation Loss: 0.00017 | Validation Accuracy: 1.00000


## PT 저장

In [13]:
# 모델 저장 코드
PATH = "B_ResNet.pt"
torch.save(model.state_dict(), PATH)

In [16]:
## PT 불러오기
# model = model()
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>